In [1]:
!pip3 install git+https://github.com/Intelligent-Systems-Lab/FedML


  Cloning https://github.com/Intelligent-Systems-Lab/FedML to /tmp/pip-req-build-cpoln5pa
  Running command git clone -q https://github.com/Intelligent-Systems-Lab/FedML /tmp/pip-req-build-cpoln5pa
  Created wheel for FedML: filename=FedML-0.0.0-py3-none-any.whl size=61594005 sha256=61ab5c5e079aa9b50d871cdfb2e59b32e02edde2d4b776fb97a78852695021ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7dfffcr/wheels/77/d7/27/9eecf324b75b35ac92998d5105c7fcbd328a851ef1086910cc
Successfully built FedML


In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 140 kB 48.0 MB/s 
     |████████████████████████████████| 180 kB 38.3 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=e6084c1661c6d047c77052070943c6f82b3e22de82f7bb62b79a320019487b31
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=a5347dd3c046cf1d63dd6b45f38012ec693e73d542dc2430a7fd476872cf0591
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import argparse
import logging
import os
import random
import sys
import time
import json
#from typing_extensions import Required
import numpy as np
import copy


import torch
import torchvision
from torchvision import transforms
from torch.utils.data import SubsetRandomSampler
import wandb

from dataloader import cifar_dataloaders

In [ ]:
from fedml_api.standalone.fedavg.fedavg_api import FedAvgAPI
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer
from model import Net, resnet56, resnet20

In [ ]:
def add_args(parser):
    """
    parser : argparse.ArgumentParser
    return a parser added with args required by fit
    """
    # Training settings
    parser.add_argument('--wandb_name', type=str, default='lab1',
                        help='Name of log file')
    
    parser.add_argument('--dataset', type=str, default='cifar10', metavar='N',
                        help='dataset used for training')

    parser.add_argument('--data_dir', type=str, default='./cifar10',
                        help='data directory')

    parser.add_argument('--batch_size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 64)')

    parser.add_argument('--client_optimizer', type=str, default='adam',
                        help='SGD with momentum; adam')

    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 0.001)')

    parser.add_argument('--wd', help='weight decay parameter;', type=float, default=0.001)

    parser.add_argument('--epochs', type=int, default=5, metavar='EP',
                        help='how many epochs will be trained locally')

    parser.add_argument('--client_num_in_total', type=int, default=10, metavar='NN',
                        help='number of workers in a distributed cluster')

    parser.add_argument('--client_num_per_round', type=int, default=10, metavar='NN',
                        help='number of workers')

    parser.add_argument('--comm_round', type=int, default=10,
                        help='how many round of communications we shoud use')

    parser.add_argument('--frequency_of_the_test', type=int, default=5,
                        help='the frequency of the algorithms')

    parser.add_argument('--gpu', type=int, default=0,
                        help='gpu')

    parser.add_argument('--ci', type=int, default=0,
                        help='CI')
                        
    parser.add_argument('--seed', type=int, default=0,
                        help='seed')
    return parser

In [ ]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

parser = add_args(argparse.ArgumentParser(description='FedAvg-standalone'))
# parser.add_argument("-f")

In [ ]:
args = parser.parse_args()
logger.info(args)
device = torch.device("cuda:" + str(args.gpu) if torch.cuda.is_available() else "cpu")
logger.info(device)

usage: ipykernel_launcher.py [-h] [--wandb_name WANDB_NAME] [--dataset N]
                             [--data_dir DATA_DIR] [--batch_size N]
                             [--client_optimizer CLIENT_OPTIMIZER] [--lr LR]
                             [--wd WD] [--epochs EP]
                             [--client_num_in_total NN]
                             [--client_num_per_round NN]
                             [--comm_round COMM_ROUND]
                             [--frequency_of_the_test FREQUENCY_OF_THE_TEST]
                             [--gpu GPU] [--ci CI] [--seed SEED]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-bb55130a-b2fc-4443-9a81-924e57d4acea.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
wandb.init(
    project="fedml",
    name=args.wandb_name,
    config=args
)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True

In [ ]:
import wandb

wandb.init(project="lab1", entity="duygu")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 172.28.0.2:9000
DEBUG:urllib3.connectionpool:http://172.28.0.2:9000 "GET /api/sessions?token= HTTP/1.1" 200 410
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: duygu (use `wandb login --relogin` to force relogin)


In [ ]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset = cifar_dataloaders(root="./cifar10", index_path="./index.json", batch_size=128, show=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
client: 0 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 1 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 2 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 3 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 4 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 5 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 6 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 7 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 8 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 9 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 10 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 11 , [250, 250, 250, 250, 250, 250, 250, 250, 250, 250], sum: 2500
client: 12 , [2

In [ ]:
model = resnet20()

In [ ]:
dummy_opt = torch.optim.SGD(copy.deepcopy(model).parameters(), lr=0.001)
dummy_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=dummy_opt, 
                                                step_size = 10/5,
                                                gamma = 0.5)
model_trainer = MyModelTrainer(model)

In [ ]:
from types import SimpleNamespace

d = {
  'epochs': 5,
  'batch_size': 128,
  'client_num_per_round': 10,
  'comm_round': 10,
  'client_num_in_total': 10,
  'client_optimizer' : 'adam',
  'lr': 0.001,
  'wd': 0.001,
  'frequency_of_the_test': 5, 
  'dataset':  'cifar10',
  'data_dir': './cifar10',
  'gpu': 0,
  'ci': 0,
  'seed': 0
}

args = SimpleNamespace(**d)

In [ ]:
args.epochs

5

In [ ]:
fedavgAPI = FedAvgAPI(dataset, device, args , model_trainer, scheduler=dummy_scheduler)

INFO:root:############setup_clients (START)#############
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:self.local_sample_number = 50000
INFO:root:############setup_clients (END)#############


In [ ]:
fedavgAPI.train()

INFO:root:################Communication round : 0
INFO:root:client_indexes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
INFO:root:Client Index = 0	Epoch: 0	Loss: 1.427630
INFO:root:Client Index = 0	Epoch: 1	Loss: 1.383669
INFO:root:Client Index = 0	Epoch: 2	Loss: 1.353133
INFO:root:Client Index = 0	Epoch: 3	Loss: 1.320830
INFO:root:Client Index = 0	Epoch: 4	Loss: 1.293594
INFO:root:Client Index = 1	Epoch: 0	Loss: 1.420495
INFO:root:Client Index = 1	Epoch: 1	Loss: 1.363849
INFO:root:Client Index = 1	Epoch: 2	Loss: 1.330737
INFO:root:Client Index = 1	Epoch: 3	Loss: 1.297606
INFO:root:Client Index = 1	Epoch: 4	Loss: 1.275348
INFO:root:Client Index = 2	Epoch: 0	Loss: 1.448472
INFO:root:Client Index = 2	Epoch: 1	Loss: 1.397231
INFO:root:Client Index = 2	Epoch: 2	Loss: 1.359619
INFO:root:Cli

# Lab2

In [ ]:
!pip3 install gdown

In [ ]:
!gdown

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--id] url_or_id
gdown: error: too few arguments


In [ ]:
import argparse
import logging
import os
import random
import sys
import time
import json
try:
    from typing_extensions import Required
except ImportError:
    from typing import Generic, TypeVar
    T = TypeVar("T")
    class Required(Generic[T]):
        pass
import numpy as np
import copy

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import SubsetRandomSampler
import wandb

from dataloader import shakespeare_dataloaders

# sys.path.insert(0, "/home/tedbest/datadisk/FedML")

from model import LSTM_shakespeare_1L
from fedml_api.standalone.fedavg.fedavg_api import FedAvgAPI


# This is answer
# from ans.shaTrainer import ShaTrainer

# yout answer
from yourTrainer import ShaTrainer


In [ ]:
if os.path.exists(os.path.abspath("./shakespeare")):
  raise ValueError("Folder: \"{}\" exist".format(os.path.abspath("./shakespeare")))
os.makedirs(os.path.abspath("./shakespeare"))
import gdown, tarfile
# download
url = 'https://drive.google.com/uc?id=1pG_tN1D874u5n3hrkkvYjKgCGWSHdF50'
output = os.path.join(os.path.abspath("./shakespeare"), 'shakespeare_all.tar.gz')
print("\nDownload ...")
gdown.download(url, output, quiet=False)
# check
md5 = '2f6538edc4202638973a9f1d8141d262'
#gdown.cached_download(url, output, md5=md5, postprocess=gdown.extractall)
time.sleep(3)
#gdown.extractall("./shakespeare/shakespeare_all.tar.gz")
# extraction
print("\nExtracting ...")
tar = tarfile.open(output, 'r:gz')
tar.extractall(path=os.path.abspath("./shakespeare"))

path = os.path.abspath("./shakespeare/shakespeare_all")

train_json_path = os.path.join(path, "train", os.listdir(os.path.join(path, "train"))[0])
test_json_path = os.path.join(path, "test", os.listdir(os.path.join(path, "test"))[0])
with open(os.path.join(train_json_path), "r") as file:
    train_context = json.load(file)
with open(os.path.join(test_json_path), "r") as file:
    test_context = json.load(file)

os.makedirs(os.path.join(os.path.abspath("."), "shakespeare"), exist_ok=True)
torch.save(train_context, os.path.join(os.path.abspath("."), "shakespeare", "train_data.pt"))
torch.save(test_context, os.path.join(os.path.abspath("."), "shakespeare", "test_data.pt"))
print("\nSave : {}".format(os.path.join(os.path.abspath("."), "shakespeare", "train_data.pt")))
print("Save : {}".format(os.path.join(os.path.abspath("."), "shakespeare", "test_data.pt")))



Download ...


Downloading...
From: https://drive.google.com/uc?id=1pG_tN1D874u5n3hrkkvYjKgCGWSHdF50
To: /content/shakespeare/shakespeare_all.tar.gz
100%|██████████| 20.0M/20.0M [00:00<00:00, 273MB/s]



Extracting ...

Save : /content/shakespeare/train_data.pt
Save : /content/shakespeare/test_data.pt


In [ ]:
from types import SimpleNamespace

d = {
  'wandb_name': 'fedavg_sha_l1_test2',
  'model': 'resnet56',
  'epochs': 1,
  'batch_size': 4,
  'client_num_per_round': 10,
  'comm_round': 80,
  'client_num_in_total': 10,
  'client_optimizer' : 'adam',
  'lr': 0.001,
  'wd': 0.001,
  'frequency_of_the_test': 1, 
  'dataset':  'cifar10',
  'data_dir': './cifar10',
  'gpu': 0,
  'ci': 1,
  'seed': 123
}

args = SimpleNamespace(**d)

In [ ]:
device = torch.device("cuda:" + str(args.gpu) if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
wandb.init(
        project="fedml",
        name=args.wandb_name,
        config=args
    )

In [ ]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True

In [ ]:
dataset = shakespeare_dataloaders(root="./shakespeare", 
                                      batch_size=args.batch_size, 
                                      clients=args.client_num_in_total)


In [ ]:
model = LSTM_shakespeare_1L()

dummy_opt = torch.optim.SGD(copy.deepcopy(model).parameters(), lr=args.lr)
dummy_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=dummy_opt, 
                                                step_size = args.comm_round/5,
                                                gamma = 0.5)
model_trainer = ShaTrainer(model)

In [ ]:
fedavgAPI = FedAvgAPI(dataset, device, args, model_trainer, scheduler=dummy_scheduler)

INFO:root:############setup_clients (START)#############
INFO:root:self.local_sample_number = 812
INFO:root:self.local_sample_number = 11048
INFO:root:self.local_sample_number = 4652
INFO:root:self.local_sample_number = 844
INFO:root:self.local_sample_number = 916
INFO:root:self.local_sample_number = 1728
INFO:root:self.local_sample_number = 4936
INFO:root:self.local_sample_number = 2384
INFO:root:self.local_sample_number = 10248
INFO:root:self.local_sample_number = 6040
INFO:root:############setup_clients (END)#############


In [ ]:
fedavgAPI.train()

INFO:root:################Communication round : 0
INFO:root:client_indexes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
INFO:root:Client Index = 0	Epoch: 0	Loss: 3.330705
INFO:root:Client Index = 1	Epoch: 0	Loss: 2.547839
INFO:root:Client Index = 2	Epoch: 0	Loss: 2.772327
INFO:root:Client Index = 3	Epoch: 0	Loss: 3.302735
INFO:root:Client Index = 4	Epoch: 0	Loss: 3.200310
INFO:root:Client Index = 5	Epoch: 0	Loss: 2.967614
INFO:root:Client Index = 6	Epoch: 0	Loss: 2.738662
INFO:root:Client Index = 7	Epoch: 0	Loss: 2.927332
INFO:root:Client Index = 8	Epoch: 0	Loss: 2.617413
INFO:root:Client Index = 9	Epoch: 0	Loss: 2.725196
INFO:root:################local_test_on_all_clients : 0
INFO:root:{'training_acc': 0.30049261083743845, 'training_loss': 2.8451580173276327}
INFO:root:{'test_acc': 0.

KeyboardInterrupt: ignored

# Lab 3 standalone

In [5]:
import argparse
import logging
import os
import random
import sys
import time
import json
try:
    from typing_extensions import Required
except ImportError:
    from typing import Generic, TypeVar
    T = TypeVar("T")
    class Required(Generic[T]):
        pass
import numpy as np
import copy

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import SubsetRandomSampler
import wandb

from dataloader import femnist_dataloaders


# from fedml_api.model.cv.resnet import resnet56
from fedml_api.standalone.fedavg.fedavg_api import FedAvgAPI
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer
from model import Net_femnist

In [6]:
from types import SimpleNamespace

d = {
  'wandb_name': 'fedavg_femnist_r20_test1',
  'model': 'resnet56',
  'epochs': 1,
  'batch_size': 16,
  'client_num_per_round': 50,
  'comm_round': 200,
  'client_num_in_total': 50,
  'client_optimizer' : 'adam',
  'lr': 0.001,
  'wd': 0.001,
  'frequency_of_the_test': 1, 
  'dataset':  'cifar10',
  'data_dir': './cifar10',
  'gpu': 0,
  'ci': 1,
  'seed': 123
}

args = SimpleNamespace(**d)

In [7]:
wandb.init(
        project="fedml",
        name=args.wandb_name,
        config=args
    )

wandb: Currently logged in as: duygu (use `wandb login --relogin` to force relogin)


In [8]:
import glob

In [9]:
if os.path.exists(os.path.abspath("./femnist")):
        raise ValueError("Folder: \"{}\" exist".format(os.path.abspath("./femnist")))
os.makedirs(os.path.abspath("./femnist"))
import gdown, tarfile
# download
url = 'https://drive.google.com/uc?id=1tzA5b92qhrFBjVTyVP645Dfs4fTV2SNL'
output = os.path.join(os.path.abspath("./femnist"), 'femnist_100up_jsonfile.tar.gz')
print("\nDownload ...")
gdown.download(url, output, quiet=False)
# check
md5 = 'df9e06065616a86228ceea8ab690012d'
#gdown.cached_download(url, output, md5=md5, postprocess=gdown.extractall)
time.sleep(3)
# extraction
print("\nExtracting ...")
tar = tarfile.open(output, 'r:gz')
tar.extractall(path=os.path.abspath("./femnist"))

path = os.path.abspath("./femnist/femnist_100up_jsonfile")

train_list = glob.glob(os.path.join(path, "train", "all_data_*.json"))
test_list = glob.glob(os.path.join(path, "test", "all_data_*.json"))

train_list = sorted(train_list, key = lambda train_list : int(train_list.split("/")[-1].split("_")[2]))
test_list = sorted(test_list, key = lambda test_list : int(test_list.split("/")[-1].split("_")[2]))

# for isl FedML tutorial, only selecte first 300 clients
train_list = train_list[:1]
test_list = test_list[:1]

train_data = {'users': [], 'num_samples': [], 'user_data': {}}
test_data = {'users': [], 'num_samples': [], 'user_data': {}}

for f in train_list:
    print("Load : {}".format(f))
    file_ = open(f, 'r')
    context = json.load(file_)
    file_.close()
    train_data['users'] += context['users']
    train_data['num_samples'] += context['num_samples']
    train_data['user_data'].update(context['user_data'])

for f in test_list:
    print("Load : {}".format(f))
    file_ = open(f, 'r')
    context = json.load(file_)
    file_.close()
    test_data['users'] += context['users']
    test_data['num_samples'] += context['num_samples']
    test_data['user_data'].update(context['user_data'])

os.makedirs(os.path.join(os.path.abspath("."), "femnist"), exist_ok=True)
torch.save(train_data, os.path.join(os.path.abspath("."), "femnist", "train_data.pt"))
print("\nSave : {}".format(os.path.join(os.path.abspath("."), "femnist", "train_data.pt")))

torch.save(test_data, os.path.join(os.path.abspath("."), "femnist", "test_data.pt"))
print("\nSave : {}".format(os.path.join(os.path.abspath("."), "femnist", "test_data.pt")))

train_data_name = {'users': copy.deepcopy(test_data['users']), }
for name in train_data["users"]:
    for i, _ in enumerate(train_data['user_data'][name]['x']):
        train_data['user_data'][name]['x'][i] = [0]
torch.save(train_data, os.path.join(os.path.abspath("."), "femnist", "train_data_name_target_only.pt"))
print("\nSave : {}".format(os.path.join(os.path.abspath("."), "femnist", "train_data_name_target_only.pt")))

time.sleep(3)
print("Done")


Download ...


Downloading...
From: https://drive.google.com/uc?id=1tzA5b92qhrFBjVTyVP645Dfs4fTV2SNL
To: /content/femnist/femnist_100up_jsonfile.tar.gz
100%|██████████| 195M/195M [00:01<00:00, 158MB/s]



Extracting ...
Load : /content/femnist/femnist_100up_jsonfile/train/all_data_0_niid_0_keep_100_train_9.json
Load : /content/femnist/femnist_100up_jsonfile/test/all_data_0_niid_0_keep_100_test_9.json

Save : /content/femnist/train_data.pt

Save : /content/femnist/test_data.pt

Save : /content/femnist/train_data_name_target_only.pt
Done


In [10]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True

In [11]:
dataset = femnist_dataloaders(root="./femnist",clients=args.client_num_in_total, batch_size=args.batch_size)


Filter clients by position.


In [12]:
model = Net_femnist()

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
dummy_opt = torch.optim.SGD(copy.deepcopy(model).parameters(), lr=args.lr)
dummy_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=dummy_opt, 
                                                step_size = args.comm_round/5,
                                                gamma = 0.5)
model_trainer = MyModelTrainer(model)

In [15]:
fedavgAPI = FedAvgAPI(dataset, device, args, model_trainer, threading=3, scheduler=dummy_scheduler)

In [16]:
fedavgAPI.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


# lab 3 standalone (4 clients)

In [23]:
from types import SimpleNamespace

d = {
  'wandb_name': 'fedavg_femnist_r20_test1_4clients',
  'model': 'resnet56',
  'epochs': 1,
  'batch_size': 16,
  'client_num_per_round': 4,
  'comm_round': 200,
  'client_num_in_total': 4,
  'client_optimizer' : 'adam',
  'lr': 0.001,
  'wd': 0.0001,
  'frequency_of_the_test': 1, 
  'dataset':  'femnist',
  'data_dir': './femnist',
  'gpu': 0,
  'ci': 1,
  'seed': 123
}

args = SimpleNamespace(**d)

In [24]:
wandb.init(
        project="fedml",
        name=args.wandb_name,
        config=args
    )

LR,▁█████████████████████████████
Test/Acc,▂▂▁▂▂▂▂▂▂▄▅▅▅▅▅▅▅▅▇▇▇▇███▇▇▇▇▇
Test/Loss,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
Train/Acc,▁▁▁▂▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█
Train/Loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
round,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
LR,3e-05
Test/Acc,0.78788
Test/Loss,1.19261
Train/Acc,0.72945
Train/Loss,1.45833


In [22]:
model = Net_femnist()
dummy_opt = torch.optim.SGD(copy.deepcopy(model).parameters(), lr=args.lr)
dummy_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=dummy_opt, 
                                                step_size = args.comm_round/5,
                                                gamma = 0.5)
model_trainer = MyModelTrainer(model)

In [25]:
fedavgAPI = FedAvgAPI(dataset, device, args, model_trainer, threading=3, scheduler=dummy_scheduler)

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [27]:
fedavgAPI.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


# Lab3 distributed (4 clients, 200 rounds)

In [31]:
!pip3 install setproctitle
!pip3 install mpi4py
!pip3 install paho-mqtt
!pip3 install -U PyYAML

     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import argparse
import logging
import os
import random
import socket
import sys
import traceback

import numpy as np
import psutil
import setproctitle
import torch
import wandb

from dataloader import femnist_dataloaders

from fedml_api.distributed.utils.gpu_mapping import mapping_processes_to_gpu_device_from_yaml_file
from fedml_api.distributed.fedavg.FedAvgAPI import FedML_init, FedML_FedAvg_distributed

from model import Net_femnist

In [2]:
from types import SimpleNamespace

d = {
  'wandb_name': 'fedavg_femnist_r20_grpc_test2',
  'model': 'resnet56',
  'epochs': 1,
  'batch_size': 16,
  'client_num_per_round': 4,
  'comm_round': 200,
  'client_num_in_total': 4,
  'client_optimizer' : 'adam',
  'lr': 0.001,
  'wd': 0.0001,
  'frequency_of_the_test': 1, 
  'dataset':  'femnist',
  'data_dir': './femnist',
  'gpu': 0,
  'ci': 1,
  'seed': 123,
  'backend': 'GRPC',
  'gpu_mapping_file' : "./grpc_config/gpu_mapping.yaml",
  'gpu_mapping_key':  "mapping_FedML_gRPC_5",
  'grpc_ipconfig_path': "./grpc_config/grpc_ipconfig.csv",
  'fl_worker_index': 0,
  'is_mobile': 1,
  'partition_alpha': 0.5,
  'partition_method': 'hetero',
  'gpu_server_num': 1,
  'gpu_num_per_server': 4,
  
}

args = SimpleNamespace(**d)

In [3]:
# customize the log format
# logging.basicConfig(level=logging.INFO,
logging.basicConfig(level=logging.DEBUG,
                    format=' - %(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                    datefmt='%a, %d %b %Y %H:%M:%S')

# parse python script input parameters
#parser = argparse.ArgumentParser()
#args = add_args(parser)
logging.info(args)

# customize the process name
str_process_name = "FedAvg (distributed):" + str(args.fl_worker_index)
setproctitle.setproctitle(str_process_name)

hostname = socket.gethostname()
logging.info("#############process ID = " + str(args.fl_worker_index) +
              ", host name = " + hostname + "########" +
              ", process ID = " + str(os.getpid()) +
              ", process Name = " + str(psutil.Process(os.getpid())))

# initialize the wandb machine learning experimental tracking platform (https://www.wandb.com/).
if args.fl_worker_index == 0:
    wandb.init(
        # project="federated_nas",
        project="fedml_rpc",
        name=args.wandb_name,
        config=args
    )

# Set the random seed. The np.random seed determines the dataset partition.
# The torch_manual_seed determines the initial weight.
# We fix these two, so that we can reproduce the result.
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

num_FL_workers = args.client_num_per_round + 1
# Please check "GPU_MAPPING.md" to see how to define the topology
device = mapping_processes_to_gpu_device_from_yaml_file(args.fl_worker_index, num_FL_workers,
                                                        args.gpu_mapping_file, args.gpu_mapping_key)

# load data
# dataset = load_data(args, args.dataset)
dataset = femnist_dataloaders(root="./femnist",clients=args.client_num_in_total, batch_size=args.batch_size)
[train_data_num, test_data_num, train_data_global, test_data_global,
  train_data_local_num_dict, train_data_local_dict, test_data_local_dict, class_num] = dataset

# create model.
# Note if the model is DNN (e.g., ResNet), the training will be very slow.
# In this case, please use our FedML distributed version (./fedml_experiments/distributed_fedavg)
model = Net_femnist()

 - Tue, 28 Dec 2021 03:37:29 <ipython-input-3-aa0be7a78b41>[line:10] INFO namespace(backend='GRPC', batch_size=16, ci=1, client_num_in_total=4, client_num_per_round=4, client_optimizer='adam', comm_round=200, data_dir='./femnist', dataset='femnist', epochs=1, fl_worker_index=0, frequency_of_the_test=1, gpu=0, gpu_mapping_file='./grpc_config/gpu_mapping.yaml', gpu_mapping_key='mapping_FedML_gRPC_5', gpu_num_per_server=4, gpu_server_num=1, grpc_ipconfig_path='./grpc_config/grpc_ipconfig.csv', is_mobile=1, lr=0.001, model='resnet56', partition_alpha=0.5, partition_method='hetero', seed=123, wandb_name='fedavg_femnist_r20_grpc_test2', wd=0.0001)
 - Tue, 28 Dec 2021 03:37:29 <ipython-input-3-aa0be7a78b41>[line:20] INFO #############process ID = 0, host name = 2628ec0190fc########, process ID = 2169, process Name = psutil.Process(pid=2169, name='FedAvg (distributed):0', started='03:37:11')
 - Tue, 28 Dec 2021 03:37:30 connectionpool.py[line:205] DEBUG Starting new HTTP connection (1): 172.28

 - Tue, 28 Dec 2021 03:37:35 gpu_mapping.py[line:22] INFO gpu_util = {'hostname_node_server': [1], 'hostname_node_1': [1, 0, 0, 0], 'hostname_node_2': [1, 0, 0, 0], 'hostname_node_3': [1, 0, 0, 0], 'hostname_node_4': [1, 0, 0, 0]}
 - Tue, 28 Dec 2021 03:37:35 gpu_mapping.py[line:31] INFO Process 0 running on host: hostname_node_server, gethostname: 2628ec0190fc, local_gpu_id: 0 ...
 - Tue, 28 Dec 2021 03:37:35 gpu_mapping.py[line:32] INFO i = 5, worker_number = 5
 - Tue, 28 Dec 2021 03:37:35 gpu_mapping.py[line:37] INFO process_id = 0, GPU device = cuda:0


Filter clients by position.


In [4]:
FedML_FedAvg_distributed(args.fl_worker_index, num_FL_workers, device, None,
                          model, train_data_num, train_data_global, test_data_global,
                          train_data_local_num_dict, train_data_local_dict, test_data_local_dict, args)

 - Tue, 28 Dec 2021 03:37:54 grpc_comm_manager.py[line:44] INFO /content
 - Tue, 28 Dec 2021 03:37:54 FedAVGAggregator.py[line:97] INFO client_indexes = [0, 1, 2, 3]


server started. Listening on port 50000


 - Tue, 28 Dec 2021 03:37:56 grpc_comm_manager.py[line:67] INFO sending message to 0.0.0.0:50001


json string size = 38088544


_InactiveRpcError: ignored